## Conv2D

In [1]:
import autoreload
%reload_ext autoreload
%autoreload 2

import sys
import os
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
sys.path.append('../src')

import tensorflow as tf
from data_processing.mtb_data_provider import MtbDataProvider
from visualization.visualize import MtbVisualizer
from models.conv1d_model import Conv1dModel
from models.conv2d_model import Conv2dModel
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.set_random_seed(RANDOM_SEED)

In [3]:
def load_data(window_size, raw_files=['neureuth'], step_size=.25, frequency=40):
    X_result = None
    y_result = None
    for raw_file in raw_files:
        X_tmp, y_tmp = MtbDataProvider.load_data('../data/raw/' + raw_file, '*?[0-9].csv', frequency=frequency)
        X_tmp = MtbDataProvider.sync_sensors(X_tmp)
        X_tmp, y_tmp = MtbDataProvider.slice_sensors(X_tmp, y_tmp, window_size=window_size, frequency=frequency, step_size=step_size)

        if X_result is None:
            X_result = X_tmp
            y_result = y_tmp
        else:
            X_result = np.concatenate((X_result, X_tmp))
            y_result = np.concatenate((y_result, y_tmp))

    return X_result, y_result, window_size // frequency

In [ ]:
def run_experiments(i=0, epochs=300, batch_size=16):
    window_sizes = [1000, 5000, 10000]
    kernel_lengths = [5, 10, 20]
    kernel_height = 2
    
    for window_size in window_sizes:
        X, y, sample_size = load_data(window_size)
        from sklearn.model_selection import train_test_split
        X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=.2, random_state=RANDOM_SEED, shuffle=True)
        
        for kernel_length in kernel_lengths:
            model_cnn2 = Conv2dModel.build_model(input_shape=X_train[0].shape, 
                                     cnn_blocks=3,
                                     dropout=0.3,
                                     kernel_size=(kernel_length, kernel_height))
            
            filepath="../checkpoints/conv2d-" + str(window_size) + "-" + str(kernel_length) + "-{epoch:02d}-{val_sparse_categorical_accuracy:.2f}.hdf5"
            
            checkpoint = ModelCheckpoint(filepath, save_best_only = True, monitor='val_sparse_categorical_accuracy', mode='max')
            early_stopping = EarlyStopping(monitor='val_sparse_categorical_accuracy', min_delta=0, patience=48, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
            
            model_cnn2.fit(X_train, y_train,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(X_dev, y_dev), 
              callbacks = [checkpoint, early_stopping])
            
            img_base_path = "../checkpoints/img/conv2d-" + str(i) + "-" + str(window_size) + "-" + str(kernel_length) + "-"
            img_path_ending = ".png"

            loss_plot_path = img_base_path  + "accuracy_plot" + img_path_ending
            plt.figure(i)
            plt.plot(model_cnn2.history.history['sparse_categorical_accuracy'])
            plt.plot(model_cnn2.history.history['val_sparse_categorical_accuracy'])
            plt.xlabel('epoch')
            plt.ylabel('sca')
            plt.savefig(loss_plot_path)
            plt.close()
            
            y_pred_dev = model_cnn2.predict(X_dev)
            y_pred_dev = np.argmax(y_pred_dev, axis=1)

            X_dev_plot = X_dev if len(X_dev.shape) <= 3 else X_dev[:,:,0]
            pred_file_path = img_base_path  + "pred_plot" + img_path_ending
            MtbVisualizer.plot_results(X_dev_plot, y_pred_dev, sample_size, pred_file_path)
            
            act_file_path = img_base_path  + "actual_plot" + img_path_ending
            MtbVisualizer.plot_results(X_dev_plot, y_dev, sample_size, act_file_path)
            
            confusion_path = img_base_path  + "confusion_matrix" + img_path_ending
            MtbVisualizer.print_confusion_matrix(y_dev, y_pred_dev, [0,1,2,3], confusion_path)


In [ ]:
for i in range(0, 20):
    run_experiments(i=i, epochs=500, batch_size=32)

reading  ../data/raw/neureuth_acc_2.csv
reading  ../data/raw/neureuth_acc_1.csv
reading  ../data/raw/neureuth_gyr_2.csv
reading  ../data/raw/neureuth_gyr_1.csv
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 4765 samples, validate on 1192 samples
Epoch 1/500
4765/4765 [==============================] - 4s 804us/sample - loss: 1.2650 - sparse_categorical_accuracy: 0.5650 - val_loss: 1.1339 - val_sparse_categorical_accuracy: 0.6099
Epoch 2/500
4765/4765 [==============================] - 1s 299us/sample - loss: 1.0296 - sparse_categorical_accuracy: 0.6558 - val_loss: 1.1354 - val_sparse_categorical_accuracy: 0.5436
Epoch 3/500
4765/4765 [==============================] - 1s 292us/sample - loss: 0.9445 - sparse_categorical_accuracy: 0.6928 - val_loss: 1.0589 - val_sparse_categorical_accuracy: 0.5831
Epoch 4/500
4765/4765 [=====================

Epoch 40/500
4765/4765 [==============================] - 1s 298us/sample - loss: 0.6061 - sparse_categorical_accuracy: 0.7870 - val_loss: 0.6228 - val_sparse_categorical_accuracy: 0.7802
Epoch 41/500
4765/4765 [==============================] - 1s 296us/sample - loss: 0.6086 - sparse_categorical_accuracy: 0.7868 - val_loss: 0.6549 - val_sparse_categorical_accuracy: 0.7559
Epoch 42/500
4765/4765 [==============================] - 1s 302us/sample - loss: 0.6114 - sparse_categorical_accuracy: 0.7807 - val_loss: 0.6448 - val_sparse_categorical_accuracy: 0.7710
Epoch 43/500
4765/4765 [==============================] - 1s 297us/sample - loss: 0.5989 - sparse_categorical_accuracy: 0.7933 - val_loss: 0.6254 - val_sparse_categorical_accuracy: 0.7878
Epoch 44/500
4765/4765 [==============================] - 1s 302us/sample - loss: 0.6112 - sparse_categorical_accuracy: 0.7847 - val_loss: 0.6310 - val_sparse_categorical_accuracy: 0.7886
Epoch 45/500
4765/4765 [==============================] - 1s

4765/4765 [==============================] - 1s 299us/sample - loss: 0.6067 - sparse_categorical_accuracy: 0.7906 - val_loss: 0.6275 - val_sparse_categorical_accuracy: 0.7668
Epoch 84/500
4765/4765 [==============================] - 1s 293us/sample - loss: 0.5976 - sparse_categorical_accuracy: 0.7918 - val_loss: 0.6573 - val_sparse_categorical_accuracy: 0.7517
Epoch 85/500
4765/4765 [==============================] - 1s 295us/sample - loss: 0.6059 - sparse_categorical_accuracy: 0.7870 - val_loss: 0.6319 - val_sparse_categorical_accuracy: 0.7710
Epoch 86/500
4765/4765 [==============================] - 1s 295us/sample - loss: 0.6031 - sparse_categorical_accuracy: 0.7874 - val_loss: 0.6184 - val_sparse_categorical_accuracy: 0.7886
Epoch 87/500
4765/4765 [==============================] - 1s 299us/sample - loss: 0.6012 - sparse_categorical_accuracy: 0.7910 - val_loss: 0.6087 - val_sparse_categorical_accuracy: 0.7936
Epoch 88/500
4765/4765 [==============================] - 1s 292us/sample

4765/4765 [==============================] - 1s 304us/sample - loss: 0.6727 - sparse_categorical_accuracy: 0.7794 - val_loss: 0.7427 - val_sparse_categorical_accuracy: 0.6820
Epoch 14/500
4765/4765 [==============================] - 1s 308us/sample - loss: 0.6634 - sparse_categorical_accuracy: 0.7801 - val_loss: 0.7426 - val_sparse_categorical_accuracy: 0.6930
Epoch 15/500
4765/4765 [==============================] - 1s 297us/sample - loss: 0.6586 - sparse_categorical_accuracy: 0.7815 - val_loss: 0.7393 - val_sparse_categorical_accuracy: 0.6720
Epoch 16/500
4765/4765 [==============================] - 2s 320us/sample - loss: 0.6568 - sparse_categorical_accuracy: 0.7809 - val_loss: 0.7127 - val_sparse_categorical_accuracy: 0.7131
Epoch 17/500
4765/4765 [==============================] - 2s 320us/sample - loss: 0.6474 - sparse_categorical_accuracy: 0.7786 - val_loss: 0.7040 - val_sparse_categorical_accuracy: 0.7232
Epoch 18/500
4765/4765 [==============================] - 1s 308us/sample

Epoch 57/500
4765/4765 [==============================] - 1s 298us/sample - loss: 0.6213 - sparse_categorical_accuracy: 0.7866 - val_loss: 0.6673 - val_sparse_categorical_accuracy: 0.7416
Epoch 58/500
4765/4765 [==============================] - 1s 301us/sample - loss: 0.6133 - sparse_categorical_accuracy: 0.7849 - val_loss: 0.6487 - val_sparse_categorical_accuracy: 0.7559
Epoch 59/500
4765/4765 [==============================] - 1s 300us/sample - loss: 0.6142 - sparse_categorical_accuracy: 0.7834 - val_loss: 0.6557 - val_sparse_categorical_accuracy: 0.7508
Epoch 60/500
4765/4765 [==============================] - 1s 306us/sample - loss: 0.6053 - sparse_categorical_accuracy: 0.7910 - val_loss: 0.6353 - val_sparse_categorical_accuracy: 0.7827
Epoch 61/500
4765/4765 [==============================] - 1s 309us/sample - loss: 0.6099 - sparse_categorical_accuracy: 0.7874 - val_loss: 0.6486 - val_sparse_categorical_accuracy: 0.7609
Epoch 62/500
4765/4765 [==============================] - 1s

4765/4765 [==============================] - 1s 297us/sample - loss: 0.6582 - sparse_categorical_accuracy: 0.7836 - val_loss: 0.6908 - val_sparse_categorical_accuracy: 0.7727
Epoch 15/500
4765/4765 [==============================] - 1s 299us/sample - loss: 0.6523 - sparse_categorical_accuracy: 0.7901 - val_loss: 0.7063 - val_sparse_categorical_accuracy: 0.7534
Epoch 16/500
4765/4765 [==============================] - 1s 297us/sample - loss: 0.6498 - sparse_categorical_accuracy: 0.7872 - val_loss: 0.6799 - val_sparse_categorical_accuracy: 0.7710
Epoch 17/500
4765/4765 [==============================] - 1s 294us/sample - loss: 0.6516 - sparse_categorical_accuracy: 0.7834 - val_loss: 0.6746 - val_sparse_categorical_accuracy: 0.7701
Epoch 18/500
4765/4765 [==============================] - 1s 288us/sample - loss: 0.6419 - sparse_categorical_accuracy: 0.7836 - val_loss: 0.6564 - val_sparse_categorical_accuracy: 0.7752
Epoch 19/500
4765/4765 [==============================] - 1s 314us/sample

Epoch 58/500
4765/4765 [==============================] - 1s 295us/sample - loss: 0.6105 - sparse_categorical_accuracy: 0.7920 - val_loss: 0.6245 - val_sparse_categorical_accuracy: 0.7953
Epoch 59/500
4765/4765 [==============================] - 1s 295us/sample - loss: 0.6133 - sparse_categorical_accuracy: 0.7906 - val_loss: 0.6327 - val_sparse_categorical_accuracy: 0.7819
Epoch 60/500
4765/4765 [==============================] - 1s 296us/sample - loss: 0.6164 - sparse_categorical_accuracy: 0.7901 - val_loss: 0.6141 - val_sparse_categorical_accuracy: 0.8012
Epoch 61/500
4765/4765 [==============================] - 1s 298us/sample - loss: 0.6137 - sparse_categorical_accuracy: 0.7893 - val_loss: 0.6324 - val_sparse_categorical_accuracy: 0.7752
Epoch 62/500
4765/4765 [==============================] - 1s 291us/sample - loss: 0.6198 - sparse_categorical_accuracy: 0.7853 - val_loss: 0.6133 - val_sparse_categorical_accuracy: 0.7987
Epoch 63/500
4765/4765 [==============================] - 1s

4765/4765 [==============================] - 1s 292us/sample - loss: 0.6074 - sparse_categorical_accuracy: 0.7899 - val_loss: 0.6576 - val_sparse_categorical_accuracy: 0.7399
Epoch 102/500
4765/4765 [==============================] - 1s 292us/sample - loss: 0.6077 - sparse_categorical_accuracy: 0.7954 - val_loss: 0.6357 - val_sparse_categorical_accuracy: 0.7802
Epoch 103/500
4765/4765 [==============================] - 1s 294us/sample - loss: 0.6132 - sparse_categorical_accuracy: 0.7843 - val_loss: 0.6919 - val_sparse_categorical_accuracy: 0.7081
Epoch 104/500
4765/4765 [==============================] - 1s 298us/sample - loss: 0.6054 - sparse_categorical_accuracy: 0.7893 - val_loss: 0.6549 - val_sparse_categorical_accuracy: 0.7534
Epoch 105/500
4765/4765 [==============================] - 1s 294us/sample - loss: 0.6005 - sparse_categorical_accuracy: 0.7916 - val_loss: 0.6389 - val_sparse_categorical_accuracy: 0.7576
Epoch 106/500
4765/4765 [==============================] - 1s 294us/s